# LINE PROFILING

**Machine Specifications:**
- Device: Dell XPS 15 9570
- OS: UBUNTU 20.04 LTS
- Memory: 16GiB System Memory, 2x 8GiB SODIMM DDR4 Synchronous 2667 MHz
- Cache:  L1 cache -> 384KiB, L2 cache -> 1536KiB, L3 cache -> 9Mib
- Swap: 20.3GiB
- Processor: Intel(R) Core(TM) i7-8750H CPU @ 2.20GHz, 6 Cores, 12 threads
- Graphic Card: Integrated -> UHD Graphics 630 Mobile, Discrete -> GeForce GTX 1050 Ti Mobile

## Importing Profilers and Lightcurve

In [1]:
import warnings, logging
import numpy as np
from stingray import Lightcurve, Crossspectrum, Powerspectrum

warnings.filterwarnings('ignore')
%load_ext line_profiler

## check_lightcurve

In [2]:
times = np.arange(100000000)
counts = np.random.rand(100000000)*100

%lprun -f Lightcurve.check_lightcurve Lightcurve(times[::-1], counts)

del times, counts

Timer unit: 1e-06 s

Total time: 15.6726 s
File: /home/apollo/stingray/stingray/lightcurve.py
Function: check_lightcurve at line 402

Line #      Hits         Time  Per Hit   % Time  Line Contents
   402                                               def check_lightcurve(self):
   403                                                   """Make various checks on the lightcurve.
   404                                           
   405                                                   It can be slow, use it if you are not sure about your
   406                                                   input data.
   407                                                   """
   408                                                   # Issue a warning if the input time iterable isn't regularly spaced,
   409                                                   # i.e. the bin sizes aren't equal throughout.
   410                                           
   411         1         47.0     47.0      0.0      

$\;\;\;\;\;\;$

## counts_err

In [3]:
def poisson_symmetrical_errors(counts):
    from astropy.stats import poisson_conf_interval
    counts_int = np.asarray(counts, dtype=np.int64)
    count_values = np.unique(counts_int)
    err_low, err_high = \
        poisson_conf_interval(count_values,
                              interval='frequentist-confidence', sigma=1)
    # calculate approximately symmetric uncertainties
    err_low -= np.asarray(count_values)
    err_high -= np.asarray(count_values)
    err = (np.absolute(err_low) + np.absolute(err_high)) / 2.0

    idxs = np.searchsorted(count_values, counts_int)
    return err[idxs]

In [4]:
def counts_err(Lightcurve):
        counts_err = Lightcurve._counts_err
        if counts_err is None and Lightcurve._countrate_err is not None:
            counts_err = Lightcurve._countrate_err * Lightcurve.dt
        elif counts_err is None:
            if Lightcurve.err_dist.lower() == 'poisson':
                counts_err = poisson_symmetrical_errors(Lightcurve.counts)
            else:
                counts_err = np.zeros_like(Lightcurve.counts)

        # If not in low-memory regime, cache the values ONLY if they have
        # been changed!
        if Lightcurve._counts_err is not counts_err:
            if not Lightcurve.low_memory or Lightcurve.input_counts:
                Lightcurve._counts_err = counts_err

        return counts_err

In [7]:
times = np.arange(100000000)
counts = np.random.rand(100000000)*100

lc = Lightcurve(times, counts, dt=1.0, skip_checks=True)

%lprun -f poisson_symmetrical_errors -f counts_err counts_err(lc)

del times, counts, lc1

Timer unit: 1e-06 s

Total time: 7.67516 s
File: <ipython-input-3-d7c5d8acecc9>
Function: poisson_symmetrical_errors at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def poisson_symmetrical_errors(counts):
     2         1      14215.0  14215.0      0.2      from astropy.stats import poisson_conf_interval
     3         1     122985.0 122985.0      1.6      counts_int = np.asarray(counts, dtype=np.int64)
     4         1    3625005.0 3625005.0     47.2      count_values = np.unique(counts_int)
     5         1          7.0      7.0      0.0      err_low, err_high = \
     6         2       3167.0   1583.5      0.0          poisson_conf_interval(count_values,
     7         1          1.0      1.0      0.0                                interval='frequentist-confidence', sigma=1)
     8                                               # calculate approximately symmetric uncertainties
     9         1          7.0    

$\;\;\;\;\;\;$

## cross_two_gtis

In [2]:
def join_equal_gti_boundaries(gti):
    new_gtis=[]
    for l in gti:
        new_gtis.append(l)
    touching = gti[:-1, 1] == gti[1:, 0]
    ng = []
    count = 0
    while count < len(gti)-1:
        if touching[count]:
            new_gtis[count+1] = [new_gtis[count][0], new_gtis[count+1][1]]
        else:
            ng.append(new_gtis[count])
        count += 1
    ng.append(new_gtis[-1])
    return np.asarray(ng)

In [3]:
def check_gtis(gti):
    gti = np.asarray(gti)
    if len(gti) < 1:
        raise ValueError("Empty GTIs")

    if len(gti) != gti.shape[0] or len(gti.shape) != 2 or \
            len(gti) != gti.shape[0]:
        raise TypeError("Please check formatting of GTIs. They need to be"
                        " provided as [[gti00, gti01], [gti10, gti11], ...]")

    gti_start = gti[:, 0]
    gti_end = gti[:, 1]

    # Check that GTIs are well-behaved
    if not np.all(gti_end >= gti_start):
        raise ValueError('This GTI end times must be larger than '
                         'GTI start times')

    # Check that there are no overlaps in GTIs
    if not np.all(gti_start[1:] >= gti_end[:-1]):
        raise ValueError('This GTI has overlaps')

    return

In [4]:
def cross_two_gtis(gti0, gti1):

    gti0 = join_equal_gti_boundaries(np.asarray(gti0))
    gti1 = join_equal_gti_boundaries(np.asarray(gti1))
    # Check GTIs
    check_gtis(gti0)
    check_gtis(gti1)

    gti0_start = gti0[:, 0]
    gti0_end = gti0[:, 1]
    gti1_start = gti1[:, 0]
    gti1_end = gti1[:, 1]

    # Create a list that references to the two start and end series
    gti_start = [gti0_start, gti1_start]
    gti_end = [gti0_end, gti1_end]

    # Concatenate the series, while keeping track of the correct origin of
    # each start and end time
    gti0_tag = np.array([0 for g in gti0_start], dtype=bool)
    gti1_tag = np.array([1 for g in gti1_start], dtype=bool)
    conc_start = np.concatenate((gti0_start, gti1_start))
    conc_end = np.concatenate((gti0_end, gti1_end))
    conc_tag = np.concatenate((gti0_tag, gti1_tag))

    # Put in time order
    order = np.argsort(conc_end)
    conc_start = conc_start[order]
    conc_end = conc_end[order]
    conc_tag = conc_tag[order]

    last_end = conc_start[0] - 1
    final_gti = []
    for ie, e in enumerate(conc_end):
        # Is this ending in series 0 or 1?
        this_series = int(conc_tag[ie])
        other_series = int(this_series == 0)

        # Check that this closes intervals in both series.
        # 1. Check that there is an opening in both series 0 and 1 lower than e
        try:
            st_pos = \
                np.argmax(gti_start[this_series][gti_start[this_series] < e])
            so_pos = \
                np.argmax(gti_start[other_series][gti_start[other_series] < e])
            st = gti_start[this_series][st_pos]
            so = gti_start[other_series][so_pos]

            s = np.max([st, so])
        except:  # pragma: no cover
            continue

        # If this start is inside the last interval (It can happen for equal
        # GTI start times between the two series), then skip!
        if s <= last_end:
            continue
        # 2. Check that there is no closing before e in the "other series",
        # from intervals starting either after s, or starting and ending
        # between the last closed interval and this one
        cond1 = (gti_end[other_series] > s) * (gti_end[other_series] < e)
        cond2 = gti_end[other_series][so_pos] < s
        condition = np.any(np.logical_or(cond1, cond2))
        # Well, if none of the conditions at point 2 apply, then you can
        # create the new gti!
        if not condition:
            final_gti.append([s, e])
            last_end = e

    return np.array(final_gti)


In [5]:
def split(Lightcurve, min_gap, min_points=1):
        # calculate the difference between time bins
        tdiff = np.diff(Lightcurve.time)
        # find all distances between time bins that are larger than `min_gap`
        gap_idx = np.where(tdiff >= min_gap)[0]

        # tolerance for the newly created GTIs: Note that this seems to work
        # with a tolerance of 2, but not if I substitute 10. I don't know why
        epsilon = np.min(tdiff)/2.0

        # calculate new GTIs
        gti_start = np.hstack([Lightcurve.time[0]-epsilon, Lightcurve.time[gap_idx+1]-epsilon])
        gti_stop = np.hstack([Lightcurve.time[gap_idx]+epsilon, Lightcurve.time[-1]+epsilon])

        gti = np.vstack([gti_start, gti_stop]).T
        if hasattr(Lightcurve, 'gti') and Lightcurve.gti is not None:
            gti = cross_two_gtis(Lightcurve.gti, gti)
        Lightcurve.gti = gti

        lc_split = Lightcurve.split_by_gti(min_points=min_points)
        return lc_split

In [6]:
times = np.arange(0, 100000000, np.random.randint(4, 9))
counts = np.random.rand(len(times))*100
    
lc = Lightcurve(times, counts, dt=1.0, skip_checks=True)

%lprun -f split -f cross_two_gtis -f join_equal_gti_boundaries -f check_gtis split(lc, 4)

del times, counts, lc

Timer unit: 1e-06 s

Total time: 45.0078 s
File: <ipython-input-2-5bc0ee67f626>
Function: join_equal_gti_boundaries at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def join_equal_gti_boundaries(gti):
     2         2          2.0      1.0      0.0      new_gtis=[]
     3  16666670    5804738.0      0.3     12.9      for l in gti:
     4  16666668    4826325.0      0.3     10.7          new_gtis.append(l)
     5         2      13926.0   6963.0      0.0      touching = gti[:-1, 1] == gti[1:, 0]
     6         2          2.0      1.0      0.0      ng = []
     7         2          0.0      0.0      0.0      count = 0
     8  16666668    6064718.0      0.4     13.5      while count < len(gti)-1:
     9  16666666    5445759.0      0.3     12.1          if touching[count]:
    10  16666666   17908796.0      1.1     39.8              new_gtis[count+1] = [new_gtis[count][0], new_gtis[count+1][1]]
    11                 

$\;\;\;\;\;\;$

## counts

In [8]:
def countFunc(Lightcurve):
    counts = Lightcurve._counts
    if Lightcurve._counts is None:
        counts = Lightcurve._countrate * Lightcurve.dt
        # If not in low-memory regime, cache the values
        if not Lightcurve.low_memory or Lightcurve.input_counts:
            Lightcurve._counts = counts

    return counts

In [10]:
times = np.arange(100000000)
counts = np.random.rand(100000000)*100

lc = Lightcurve(times, counts, dt=1.0, skip_checks=True)

%lprun -f countFunc countFunc(lc)

del times, counts, lc

Timer unit: 1e-06 s

Total time: 2e-06 s
File: <ipython-input-8-8139c4bf5cf9>
Function: countFunc at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def countFunc(Lightcurve):
     2         1          2.0      2.0    100.0      counts = Lightcurve._counts
     3         1          0.0      0.0      0.0      if Lightcurve._counts is None:
     4                                                   counts = Lightcurve._countrate * Lightcurve.dt
     5                                                   # If not in low-memory regime, cache the values
     6                                                   if not Lightcurve.low_memory or Lightcurve.input_counts:
     7                                                       Lightcurve._counts = counts
     8                                           
     9         1          0.0      0.0      0.0      return counts

$\;\;\;\;\;\;$

## _ getitem _

In [4]:
times = np.arange(100000000)
counts = np.random.rand(100000000)*100

lc = Lightcurve(times, counts, dt=1.0, skip_checks=True)

%lprun -f lc.__getitem__ lc.__getitem__(slice(10000, 10000000))

del times, counts, lc

Timer unit: 1e-06 s

Total time: 0.000308 s
File: /home/apollo/stingray/stingray/lightcurve.py
Function: __getitem__ at line 628

Line #      Hits         Time  Per Hit   % Time  Line Contents
   628                                               def __getitem__(self, index):
   629                                                   """
   630                                                   Return the corresponding count value at the index or a new :class:`Lightcurve`
   631                                                   object upon slicing.
   632                                           
   633                                                   This method adds functionality to retrieve the count value at
   634                                                   a particular index. This also can be used for slicing and generating
   635                                                   a new :class:`Lightcurve` object. GTIs are recalculated based on the new light
   636            

$\;\;\;\;\;\;$

## sort_counts

In [2]:
times = np.arange(10000000)
counts = np.random.rand(10000000)*100

lc = Lightcurve(times, counts[::-1], dt=1.0, skip_checks=True)

%lprun -f lc.sort_counts lc.sort_counts()

del times, counts, lc

Timer unit: 1e-06 s

Total time: 43.0702 s
File: /home/apollo/stingray/stingray/lightcurve.py
Function: sort_counts at line 1210

Line #      Hits         Time  Per Hit   % Time  Line Contents
  1210                                               def sort_counts(self, reverse=False):
  1211                                                   """
  1212                                                   Sort a :class:`Lightcurve` object in accordance with its counts array.
  1213                                           
  1214                                                   A :class:`Lightcurve` can be sorted in either increasing or decreasing order
  1215                                                   using this method. The counts array gets sorted and the time array is
  1216                                                   changed accordingly.
  1217                                           
  1218                                                   Parameters
  1219              

$\;\;\;\;\;\;$

## join

In [2]:
times = np.arange(10000000)
counts = np.random.rand(10000000)*100

lc1 = Lightcurve(times, counts, dt=1.0, skip_checks=True)
lc2 = Lightcurve(times, counts[::-1], dt=1.0, skip_checks=True)

%lprun -f lc1.join lc1.join(lc2)

del times, counts, lc1, lc2

Timer unit: 1e-06 s

Total time: 149.834 s
File: /home/apollo/stingray/stingray/lightcurve.py
Function: join at line 891

Line #      Hits         Time  Per Hit   % Time  Line Contents
   891                                               def join(self, other):
   892                                                   """
   893                                                   Join two lightcurves into a single object.
   894                                           
   895                                                   The new :class:`Lightcurve` object will contain time stamps from both the
   896                                                   objects. The ``counts`` and ``countrate`` attributes in the resulting object
   897                                                   will contain the union of the non-overlapping parts of the two individual objects,
   898                                                   or the average in case of overlapping ``time`` arrays of both :cla

$\;\;\;\;\;\;$

## _fourier_cross

In [2]:
times = np.arange(10000000)
counts = np.random.rand(10000000)*100

lc1 = Lightcurve(times, counts, dt=1.0, skip_checks=True)
lc2 = Lightcurve(times, counts[::-1], dt=1.0, skip_checks=True)

cspec = Crossspectrum(lc1, lc2)

%lprun -f cspec._fourier_cross cspec._fourier_cross(lc1, lc2)

del times, counts, lc1, lc2, cspec

Timer unit: 1e-06 s

Total time: 0.612443 s
File: /home/apollo/stingray/stingray/crossspectrum.py
Function: _fourier_cross at line 561

Line #      Hits         Time  Per Hit   % Time  Line Contents
   561                                               def _fourier_cross(self, lc1, lc2):
   562                                                   """
   563                                                   Fourier transform the two light curves, then compute the cross spectrum.
   564                                                   Computed as CS = lc1 x lc2* (where lc2 is the one that gets
   565                                                   complex-conjugated)
   566                                           
   567                                                   Parameters
   568                                                   ----------
   569                                                   lc1: :class:`stingray.Lightcurve` object
   570                                     

$\;\;\;\;\;\;$

## rebin_data 

In [2]:
def apply_function_if_none(variable, value, func):
    if variable is None:
        return func(value)
    else:
        return variable

In [3]:
def rebin_data(x, y, dx_new, yerr=None, method='sum', dx=None):
    y = np.asarray(y)
    yerr = np.asarray(apply_function_if_none(yerr, y, np.zeros_like))

    dx_old = apply_function_if_none(dx, np.diff(x), np.median)

    if dx_new < dx_old:
        raise ValueError("New frequency resolution must be larger than "
                         "old frequency resolution.")

    step_size = dx_new / dx_old

    output = []
    outputerr = []
    for i in np.arange(0, y.shape[0], step_size):
        total = 0
        totalerr = 0

        int_i = int(i)
        prev_frac = int_i + 1 - i
        prev_bin = int_i
        total += prev_frac * y[prev_bin]
        totalerr += prev_frac * (yerr[prev_bin] ** 2)

        if i + step_size < len(x):
            # Fractional part of next bin:
            next_frac = i + step_size - int(i + step_size)
            next_bin = int(i + step_size)
            total += next_frac * y[next_bin]
            totalerr += next_frac * (yerr[next_bin] ** 2)

        total += sum(y[int(i + 1):int(i + step_size)])
        totalerr += sum(yerr[int(i + 1):int(step_size)] ** 2)
        output.append(total)
        outputerr.append(np.sqrt(totalerr))

    output = np.asarray(output)
    outputerr = np.asarray(outputerr)

    if method in ['mean', 'avg', 'average', 'arithmetic mean']:
        ybin = output / np.float(step_size)
        ybinerr = outputerr / np.sqrt(np.float(step_size))

    elif method == "sum":
        ybin = output
        ybinerr = outputerr

    else:
        raise ValueError("Method for summing or averaging not recognized. "
                         "Please enter either 'sum' or 'mean'.")

    tseg = x[-1] - x[0] + dx_old

    if (tseg / dx_new % 1) > 0:
        ybin = ybin[:-1]
        ybinerr = ybinerr[:-1]

    new_x0 = (x[0] - (0.5 * dx_old)) + (0.5 * dx_new)
    xbin = np.arange(ybin.shape[0]) * dx_new + new_x0

    return xbin, ybin, ybinerr, step_size


In [4]:
 def rebin(cspec, df=None, f=None, method="mean"):
        if f is None and df is None:
            raise ValueError('You need to specify at least one between f and '
                             'df')
        elif f is not None:
            df = f * cspec.df

        # rebin cross spectrum to new resolution
        binfreq, bincs, binerr, step_size = \
            rebin_data(cspec.freq, cspec.power, df, cspec.power_err,
                       method=method, dx=cspec.df)

        # make an empty cross spectrum object
        # note: syntax deliberate to work with subclass Powerspectrum
        bin_cs = copy.copy(cspec)

        # store the binned periodogram in the new object
        bin_cs.freq = binfreq
        bin_cs.power = bincs
        bin_cs.df = df
        bin_cs.n = cspec.n
        bin_cs.norm = cspec.norm
        bin_cs.nphots1 = cspec.nphots1
        bin_cs.power_err = binerr

        if hasattr(cspec, 'unnorm_power'):
            _, binpower_unnorm, _, _ = \
                rebin_data(cspec.freq, cspec.unnorm_power, df,
                           method=method, dx=cspec.df)

            bin_cs.unnorm_power = binpower_unnorm

        if hasattr(cspec, 'cs_all'):
            cs_all = []
            for c in cspec.cs_all:
                cs_all.append(c.rebin(df=df, f=f, method=method))
            bin_cs.cs_all = cs_all
        if hasattr(cspec, 'pds1'):
            bin_cs.pds1 = cspec.pds1.rebin(df=df, f=f, method=method)
        if hasattr(cspec, 'pds2'):
            bin_cs.pds2 = cspec.pds2.rebin(df=df, f=f, method=method)

        try:
            bin_cs.nphots2 = cspec.nphots2
        except AttributeError:
            if cspec.type == 'powerspectrum':
                pass
            else:
                raise AttributeError(
                    'Spectrum has no attribute named nphots2.')

        bin_cs.m = np.rint(step_size * cspec.m)

        return bin_cs

In [5]:
import copy
times = np.arange(10000000)
counts = np.random.rand(10000000)*100

lc1 = Lightcurve(times, counts, dt=1.0, skip_checks=True)
lc2 = Lightcurve(times, counts[::-1], dt=1.0, skip_checks=True)

cspec = Crossspectrum(lc1, lc2)

%lprun -f rebin_data rebin(cspec, df=2.0)

del times, counts, lc1, lc2, cspec

Timer unit: 1e-06 s

Total time: 2.63773 s
File: <ipython-input-3-7798820b31f4>
Function: rebin_data at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def rebin_data(x, y, dx_new, yerr=None, method='sum', dx=None):
     2         2          7.0      3.5      0.0      y = np.asarray(y)
     3         2       8958.0   4479.0      0.3      yerr = np.asarray(apply_function_if_none(yerr, y, np.zeros_like))
     4                                           
     5         2      12289.0   6144.5      0.5      dx_old = apply_function_if_none(dx, np.diff(x), np.median)
     6                                           
     7         2          6.0      3.0      0.0      if dx_new < dx_old:
     8                                                   raise ValueError("New frequency resolution must be larger than "
     9                                                                    "old frequency resolution.")
    10      

$\;\;\;\;\;\;$

## rms_error

In [6]:
import scipy.stats
def _rms_error1(pspec, powers):
    nphots = pspec.nphots
    p_err = scipy.stats.chi2(2.0 * pspec.m).var() * powers / pspec.m / nphots

    rms = np.sum(powers) / nphots
    pow = np.sqrt(rms)

    drms_dp = 1 / (2 * pow)

    sq_sum_err = np.sqrt(np.sum(p_err**2))
    delta_rms = sq_sum_err * drms_dp
    return delta_rms

In [7]:
def compute_rms(pspec, min_freq, max_freq, white_noise_offset=0.):
    minind = pspec.freq.searchsorted(min_freq)
    maxind = pspec.freq.searchsorted(max_freq)
    powers = pspec.power[minind:maxind]
    nphots = pspec.nphots

    if pspec.norm.lower() == 'leahy':
        powers_leahy = powers.copy()
    elif pspec.norm.lower() == "frac":
        powers_leahy = \
            pspec.unnorm_power[minind:maxind].real * 2 / nphots
    else:
        raise TypeError("Normalization not recognized!")

    rms = np.sqrt(np.sum(powers_leahy - white_noise_offset) / nphots)
    rms_err = _rms_error1(pspec, powers_leahy)

    return rms, rms_err

In [8]:
times = np.arange(10000000)
counts = np.random.rand(10000000)*100

lc = Lightcurve(times, counts, dt=1.0, skip_checks=True)
pspec = Powerspectrum(lc, norm='leahy')

%lprun -f _rms_error1 compute_rms(pspec, min_freq=0.001, max_freq=0.499)

del times, counts, lc, pspec

Timer unit: 1e-06 s

Total time: 0.023109 s
File: <ipython-input-6-307575921b57>
Function: _rms_error1 at line 2

Line #      Hits         Time  Per Hit   % Time  Line Contents
     2                                           def _rms_error1(pspec, powers):
     3         1          2.0      2.0      0.0      nphots = pspec.nphots
     4         1      13076.0  13076.0     56.6      p_err = scipy.stats.chi2(2.0 * pspec.m).var() * powers / pspec.m / nphots
     5                                           
     6         1       2065.0   2065.0      8.9      rms = np.sum(powers) / nphots
     7         1          7.0      7.0      0.0      pow = np.sqrt(rms)
     8                                           
     9         1          3.0      3.0      0.0      drms_dp = 1 / (2 * pow)
    10                                           
    11         1       7954.0   7954.0     34.4      sq_sum_err = np.sqrt(np.sum(p_err**2))
    12         1          1.0      1.0      0.0      delta_rms = s

$\;\;\;\;\;\;$

## gti_border_bins

In [2]:
def gti_border_bins(gtis, time, dt=None, epsilon=0.001):
    if dt is None:
        dt = np.median(np.diff(time))

    epsilon_times_dt = epsilon * dt
    spectrum_start_bins = np.array([], dtype=np.long)
    spectrum_stop_bins = np.array([], dtype=np.long)
    for g in gtis:
        good = (time - dt / 2 >= g[0]) & (time + dt / 2 <= g[1])
        t_good = time[good]
        if len(t_good) == 0:
            continue
        startbin = np.argmin(np.abs(time - dt / 2 - g[0]))
        stopbin = np.searchsorted(time + dt / 2, g[1], 'right') + 1
        if stopbin > len(time):
            stopbin = len(time)

        if time[startbin] < (g[0] + dt / 2 - epsilon_times_dt):
            startbin += 1
        # Would be g[1] - dt/2, but stopbin is the end of an interval
        # so one has to add one bin
        if time[stopbin - 1] > (g[1] - dt / 2 + epsilon_times_dt):
            stopbin -= 1
        spectrum_start_bins = \
            np.append(spectrum_start_bins,
                      [startbin])
        spectrum_stop_bins = \
            np.append(spectrum_stop_bins,
                      [stopbin])
    assert len(spectrum_start_bins) > 0, \
        ("No GTIs are equal to or longer than chunk_length.")
    return spectrum_start_bins, spectrum_stop_bins

In [3]:
times = np.arange(10000000)
counts = np.random.rand(10000000)*100

lc = Lightcurve(times, counts, dt=1.0, skip_checks=True)

%lprun -f gti_border_bins gti_border_bins(lc.gti, lc.time, lc.dt)

del times, counts, lc

Timer unit: 1e-06 s

Total time: 0.124913 s
File: <ipython-input-2-ae32d5e7cef5>
Function: gti_border_bins at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def gti_border_bins(gtis, time, dt=None, epsilon=0.001):
     2         1          1.0      1.0      0.0      if dt is None:
     3                                                   dt = np.median(np.diff(time))
     4                                           
     5         1          0.0      0.0      0.0      epsilon_times_dt = epsilon * dt
     6         1          3.0      3.0      0.0      spectrum_start_bins = np.array([], dtype=np.long)
     7         1          2.0      2.0      0.0      spectrum_stop_bins = np.array([], dtype=np.long)
     8         2          8.0      4.0      0.0      for g in gtis:
     9         1      44724.0  44724.0     35.8          good = (time - dt / 2 >= g[0]) & (time + dt / 2 <= g[1])
    10         1      17851.0  17851

$\;\;\;\;\;\;$